In [3]:
import requests 
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import platform
from ultralytics import YOLO
import io
import base64
from gradio import ChatMessage
import re
import time
import azure.cognitiveservices.speech as speechsdk

last_gpt_request_time = 0  # 마지막 GPT 요청 시간 기록
gpt_request_interval = 10  # GPT 요청 간격 (초)
model = YOLO('yolo11x.pt').to('cpu')
SPEECH_ENDPOINT = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"
SPEECH_API_KEY = "6d450cc0d7e64602a2a9c302854fce19"

# open AI request 받아오는 함수
def request_gpt(prompt, histories=[]):
    ai_search_endpoint = "https://aisearch-team8-eastus.search.windows.net"
    ai_search_key = "hjEB318eEZKXt5im3PM1ejZ1HXcZfAKoO3ygcC7YVXAzSeANXLCQ"
    ai_search_index = "metro2"
    endpoint = "https://openai-team8-eastus2.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-type": "application/json",
        "api-key": "8NSOlHOOaVXr898vFkT3m8vMuNsjYoMYwIvw0zl8uRHxok0M0Jv7JQQJ99ALACHYHv6XJ3w3AAABACOGlhnZ"
    }
    
    message_list = list()
    
    # 시스템 메시지
    message_list.append({
        "role": "system",
        "content": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다."       
        
    })

    # 이전 메시지 포함.
    for history in histories:
        for text in history:
            message_list.append({
                "role": "assistant",
                "content": [
                    {
                    "type": "text",
                    "text": text
                    }
                ]
            })

    # 유저 메시지.
    message_list.append({
        "role": "user",
        "content": prompt
    })            
    
    payload = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "semantic_configuration": "metro2-semantic",
                "query_type": "semantic",
                "strictness": 2,
                "top_n_documents": 5,
                "key": ai_search_key,
                "indexName": ai_search_index
            }
        }
        ],
    }  
    
    response = requests.post(endpoint, headers=headers, json=payload)
    print(response.status_code)
    if response.status_code == 200:
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]

        if response_json["choices"][0]["message"]["context"]:
            citations = response_json["choices"][0]["message"]["context"]["citations"]
            formatted_citation_list = list()
            i = 0
            for c in citations:
                i += 1
                temp = f"<details><summary>참조{i}</summary><ul>{c['content']}</ul></details>"
                formatted_citation_list.append(temp)

        else:
            # citations = ""
            formatted_citation_list = list()
        return content, "".join(formatted_citation_list)
    else:
        return f"{response.status_code}, {response.text}", ""


def request_yolo_gpt(image_array, objects):
    endpoint = "https://openai-team8-eastus2.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": "8NSOlHOOaVXr898vFkT3m8vMuNsjYoMYwIvw0zl8uRHxok0M0Jv7JQQJ99ALACHYHv6XJ3w3AAABACOGlhnZ"
    }

    # numpy 이미지를 PIL 형태로 변환
    image = Image.fromarray(image_array)
    buffered_io = io.BytesIO()
    image.save(buffered_io, format='png')
    base64_image = base64.b64encode(buffered_io.getvalue()).decode('utf-8')

    object_descriptions = "\n".join([
    f"{obj[0]} ({obj[2]*100:.2f}%) at {obj[1]}" for obj in objects
])
    
    user_message = f"""
    너는 물체를 감지해서 시각장애인에게 앞에 무엇이 있는지 위험을 알려주는 YOLO 모델이야.
    감지된 물체는 다음과 같아:
    {object_descriptions}
    이 사진에서 바운딩 박스에 감지된 물체에 대해 왜 위험한지 간단하게 설명해줘.
    """

    message_list = [
        {
            "role": "system",
            "content": "너는 사진 속에서 감지된 물체에 대해 위험성을 분석하는 봇이야."
        }
    ]

    message_list.append({
        "role": "user",
        "content": [{
            "type": "text",
            "text": user_message
        },{
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
        }]
    })

    payload = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1500
    }
    response = requests.post(endpoint, headers=headers, json=payload)
    if response.status_code == 200:
        response_json = response.json()
        return response_json['choices'][0]['message']['content']
    else:
        return "Error: Unable to process your request."
    
def request_tts(text):
    endpoint = SPEECH_ENDPOINT
    headers = {
        "Ocp-Apim-Subscription-Key": SPEECH_API_KEY,
        "Content-Type": "application/ssml+xml",
        "X-Microsoft-OutputFormat": "audio-16khz-128kbitrate-mono-mp3"
    }
    payload = f"""
    <speak version='1.0' xml:lang='en-US'>
        <voice xml:lang='ko-KR' xml:gender='Female' name='ko-KR-SunHiNeural'>
            {text}
        </voice>
    </speak>
    """
    
    response = requests.post(endpoint, headers=headers, data=payload)

    if response.status_code == 200:
        file_name = "response_audio.mp3"
        print(response.status_code)
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        return file_name
    else:
        return None

def detect_objects(origin_image):
    image = origin_image.copy()
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    response = model(origin_image)

    font_size = 15
    if platform.system() == "Darwin":
        font = ImageFont.truetype("AppleGothic.ttf", size=font_size)
    elif platform.system() == "Windows":
        font = ImageFont.truetype("malgun.ttf", size=font_size)
    else:
        font = ImageFont.load_default(size=font_size)

    filtered_results = []  # 필터링된 결과 저장

    for result in response:
        label_list = result.names
        box_list = result.boxes.xyxy.cpu().numpy()
        confidence_list = result.boxes.conf.cpu().numpy()
        class_id_list = result.boxes.cls.cpu().numpy()

        for index, box in enumerate(box_list):
            confidence = confidence_list[index]
            class_id = class_id_list[index]
            label = label_list[class_id]

            # 라벨링 데이터만 필터링 (예: 신뢰도 50% 이상만 포함)
            if confidence >= 0.5:  # 조건에 따라 필터링 가능
                filtered_results.append((label, box, confidence))

                x1, y1, x2, y2 = map(int, box)
                draw.rectangle((x1, y1, x2, y2), outline=(255, 217, 0), width=2)
                draw.text((x1 + 5, y1 + 5), text=f"{label} ({(confidence * 100):.2f}%)", fill=(255, 217, 0), font=font)

    return image, filtered_results  # 이미지와 필터링된 결과 반환

def request_stt_fast(file_path):
    endpoint = "https://eastus.api.cognitive.microsoft.com/speechtotext/transcriptions:transcribe?api-version=2024-11-15"
    headers = {
        "Ocp-Apim-Subscription-Key": "6d450cc0d7e64602a2a9c302854fce19",
    }

    with open(file_path, "rb") as audio :
        audio_data = audio.read()

    json = {
        "definition": '{"locales":["ko-KR"], "profanityFilterMode":"Masked", "channels":[0,1]}'
    }

    files = {
        "audio" : audio_data
    }

    response = requests.post(endpoint, headers=headers,files=files, json=json)

    print("Fast", response.status_code, response.elapsed.total_seconds())
    if response.status_code == 200:
        response_json = response.json()
        response_text = response_json['combinedPhrases'][0]['text']
        print(response_text)
        return response_text
    else:
        return ''

def change_chatbot(histories):
    content = histories[-1]['content']

    pattern = r'[^가-힣a-zA-Z0-9\s%,\.]'
    cleaned_text = re.sub(pattern, '', content)
    file_name = request_tts(cleaned_text)
    
    return file_name

chat_history = []  # 채팅 히스토리 기록

def stream_webcam(image):
    global last_gpt_request_time, chat_history
    current_time = time.time()

    # 객체 감지
    detected_image, detected_objects = detect_objects(image)

    # GPT 요청 주기적 실행
    if current_time - last_gpt_request_time > gpt_request_interval:
        last_gpt_request_time = current_time
        gpt_response = request_yolo_gpt(image_array=image, objects=detected_objects)

        # 채팅 기록에 GPT 응답 추가
        chat_history.append({"role": "assistant", "content": gpt_response})

    return detected_image, chat_history

def change_audio(file_path):
    if file_path:
        response_text = request_stt_fast(file_path=file_path)
        return response_text
    else:
        return ''
    
def click_send(prompt, histories):
    response_text, citation_html = request_gpt(prompt=prompt, histories=histories)
    histories.append((prompt, response_text))
    
    pattern = r'[^가-힣a-zA-Z0-9\s]'
    formatted_response_text = re.sub(pattern, '', response_text)
    
    audio_file_name = request_tts(formatted_response_text)
    return histories, audio_file_name, citation_html

def synthesize_speech(text):
    speech_key = "4cc73b76c3984f13a64e8230fc8b5d8d" 
    region = "eastus"
    
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)  # 기본 스피커로 음성 출력

    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    synthesizer.speak_text_async(text) 

def read_tab_label(tab_label):
        """탭의 라벨을 읽어서 TTS로 출력"""
        synthesize_speech(tab_label)

In [4]:
with gr.Blocks() as demo:
    
    with gr.Tab(label="위험 감지") as tab1 :
        gr.Markdown("## 📸 위험 감지 화면")

        # 실시간 화면과 검출 화면을 세로 배치
        with gr.Column():
            input_webcam = gr.Image(
                label="실시간 화면",
                sources="webcam",
                height=250,  # 모바일 화면 최적화
                width="100%",
                mirror_webcam=False
            )

        gr.Markdown("### 검출 화면 및 분석 결과")
        with gr.Row():
            output_image = gr.Image(
                label="검출 화면",
                type="pil",
                interactive=False,
                height=220,
                width="50%"
            )

            chatbot_yolo = gr.Chatbot(
                label="⚠️ 분석 결과",
                height=220,  # 모바일 화면에 맞는 높이
                type="messages"
            )          

        # 분석 결과
        with gr.Column():

            gr.Markdown("### 🔊 분석 오디오")
            chatbot_audio_yolo = gr.Audio(
                label="🔊 분석 오디오",
                interactive=False,
                autoplay=True, waveform_options=gr.WaveformOptions(
                    waveform_color="#D3B1C2",
                    waveform_progress_color="#C197D@",
                    skip_length=2,
                    show_controls=False))

        # 스트림 함수 연결
        input_webcam.stream(
            fn=stream_webcam,
            inputs=[input_webcam],
            outputs=[output_image, chatbot_yolo]
        )
        chatbot_yolo.change(fn=change_chatbot, inputs=[chatbot_yolo], outputs=[chatbot_audio_yolo])

    with gr.Tab(label="교통 정보")  as tab2:
        gr.Markdown("## 🚦 교통 정보")

        # 음성 입력 및 결과 출력
        with gr.Column():
            gr.Markdown("### 🎤 마이크 입력")
            input_mic = gr.Audio(
                label="🎤 마이크 입력",
                sources="microphone",
                type="filepath",
                interactive=True, waveform_options=gr.WaveformOptions(
                    waveform_color="#D3B1C2",
                    waveform_progress_color="#C197D@",
                    skip_length=2,
                    show_controls=False)
            )
            input_openai_textbox = gr.Textbox(
                label="📝 텍스트", 
                interactive=False, 
                placeholder="음성 입력 결과가 여기에 표시됩니다."
            )

        # 프롬프트 입력과 결과
        gr.Markdown("### 💬 대화 내용")
        chatbot_gpt = gr.Chatbot(
            label="💬 대화 내용", 
            height=200
        )
        gr.Markdown("### 🔊 GPT 음성 응답")
        chatbot_audio_gpt = gr.Audio(
            label="🔊 GPT 음성 응답", 
            interactive=False,
            autoplay=True, 
        )
        citation = gr.HTML(label="🔗 참조 링크")

        # 이벤트 연결
        input_mic.change(
            fn=change_audio, 
            inputs=[input_mic], 
            outputs=[input_openai_textbox]
        )
        input_openai_textbox.change(
            fn=click_send,
            inputs=[input_openai_textbox, chatbot_gpt],
            outputs=[chatbot_gpt, chatbot_audio_gpt, citation]
        )
        
        # 페이지 로드시 기본으로 위험 감지 값을 가지고 tts로 출력
    demo.load(
        fn=read_tab_label,
        inputs=gr.State(value="위험 감지"),  # 초기 탭 라벨 설정
            )
    # 탭 선택 시 어떤 탭인지 tts 출력
    tab1.select(
        fn=read_tab_label,
        inputs=gr.State(value="위험 감지")
               )
    tab2.select(
        fn=read_tab_label,
        inputs=gr.State(value="교통 정보")
               )

demo.launch(share=True)

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://fa751d4f02d613961f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 384x640 1 person, 3380.2ms
Speed: 11.5ms preprocess, 3380.2ms inference, 18.5ms postprocess per image at shape (1, 3, 384, 640)

200
0: 384x640 1 person, 2673.2ms
Speed: 3.9ms preprocess, 2673.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

200
0: 384x640 1 person, 4530.8ms
Speed: 7.7ms preprocess, 4530.8ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3077.8ms
Speed: 3.4ms preprocess, 3077.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
